In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [159]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [160]:
# cat = Table.read('./Cutout/NSA/nsa_cutout_cat_1103.fits')
# cat = Table.read('./Catalog/nsa_20hosts_sample_211103_visual_inspect.fits')
cat = Table.read('./Catalog/20hosts/after_size_color_sb_cas_cut_jg_large_A.fits')
cat.rename_column('ID', 'viz-id')
cat['viz-id'] = cat['viz-id'].astype(int)

In [161]:
# cat = Table.read('./Catalog/nsa_20hosts_sample_211103_visual_inspect.fits')

In [162]:
# cat[cat['viz-id'] == 14399]

In [163]:
# ind = np.array([ 1346.,  2154.,  2676.,  4150.,  4414.,  5794.,  8700., 13702.,
#        13964., 14260., 14299., 17929., 18167., 19532., 22478., 22486.,
#        23059., 23522., 23781., 24028., 24816., 26176., 27243., 28405.,
#        32582., 35399., 36300., 42145., 42524., 43465., 45091., 45917.,
#        46387., 46552., 46635., 47487., 50161., 50626., 50763., 51205.,
#        52184., 52277., 52718., 53316., 56488., 57472., 57562., 60266.,
#        63010., 64518., 65190., 67699., 68632., 69320., 70027., 70553.,
#        71082., 71842., 72424., 72687., 73208., 73274., 74301., 74345.,
#        75236., 76316., 77062., 77194., 79520., 81401., 84178., 85340.,
#        86601., 86770., 88255.]).astype(int)

In [164]:
junk = cat['junk']
candy = cat['candy']
gal = cat['gal']

In [165]:
print('Candy:', np.sum(candy))
print('Gal:', np.sum(gal))
print('Junk:', np.sum(junk))

Candy: 57
Gal: 17
Junk: 2


In [127]:
# cat = cat[np.in1d(cat['viz-id'], ind)] # junks only
# len(cat)

In [166]:
cat.sort('viz-id')

In [167]:
cat = cat#[candy]
category = 'large_A'

### Generate RGB images

In [168]:
def _display_cutout_rgb(images, ax=None, half_width=200):
    import scarlet
    from scarlet.display import AsinhMapping

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
    
    # Crop
    if half_width is not None:
        if half_width * 2 < min(images.shape[1], images.shape[2]):
            cen = (images.shape[1] // 2, images.shape[2] // 2)
            images = images[:, cen[0] - half_width:cen[0]+half_width, cen[1] - half_width:cen[1] + half_width]
    
    # Norm color
    f_c = np.array([1.9, 1.2, 1., 0.85])
    _images = images * f_c[:, np.newaxis, np.newaxis]
    
    # Display
    norm = AsinhMapping(minimum=-0.15, stretch=1.2, Q=3)
    
    img_rgb = scarlet.display.img_to_rgb(_images, norm=norm)
    plt.imshow(img_rgb, origin='lower')
    ax.axis('off')
    
    if ax is None:
        return fig
    return ax

In [169]:
# obj = cat[cat['viz-id'] == 68320][0]
# ind = obj['viz-id']

# images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
# fig, ax = plt.subplots(figsize=(5, 5))
# size = obj['rhalf_ellip'] * 7 if obj['rhalf_ellip'] * 5 > 100 else 100
# ax = _display_cutout_rgb(images, ax=ax, half_width=int(size))
# #plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
# #plt.close()

In [170]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/viz_ins/'
WEBPAGE_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/20hosts/{category}/'

In [171]:
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            
if not os.path.isdir(FIGURE_DIR):
    os.mkdir(FIGURE_DIR)

In [172]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        size = obj['rhalf_ellip'] * 5 if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = _display_cutout_rgb(images, ax=ax, half_width=int(size))
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 76
Progress: 10 / 76
Progress: 20 / 76
Progress: 30 / 76
Progress: 40 / 76
Progress: 50 / 76
Progress: 60 / 76
Progress: 70 / 76


In [173]:
# Copy figures in FIGURE_DIR to WEBPAGE_DIR
if not os.path.isdir(WEBPAGE_DIR):
    os.mkdir(WEBPAGE_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

### Generate webpage for cutouts. WE use `diezi/webpage/gen_web_cutout.py`

In [174]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [175]:
#FOR NSA SAMPLE
FIGURE_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/20hosts/{category}/'
WEBPAGE_DIR = f'/tigress/jiaxuanl/public_html/NSA/cutout_RGB/20hosts/{category}/'
SCARLET_DIR = '/tigress/jiaxuanl/public_html/NSA/vanilla_result/'
sample_name = 'NSA'

In [176]:
webpage_cutout(FIGURE_DIR, WEBPAGE_DIR, sample_name, sample_title=f'{category}', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 76 galaxies to be displayed
Total pages: 4


In [177]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/20hosts/large_A/


### The search box on webpage still doesn't work. Need to work on the javascript file